In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../src_jobs/')

In [2]:
import pickle
from itertools import repeat
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from artifact import Saw_centered, Saw_centered_Francois
from sliding_window_detector import SlidingWindowTransformerDetector, SlidingWindowLinearDetector, FineTunedSlidingWindowDetector, ConvolutionalSlidingWindowDetector

from data import CachedArtifactDataset, TestArtifactDataset, CenteredArtifactDataset

/opt/miniconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
torch.set_grad_enabled(False)

In [7]:
test_width_512 = 512
data_512 = pd.read_pickle('/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/real/francois_normalized_dataset.pickle') 

test_width_256 = 256
data_256 = pd.read_pickle('/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/real/francois_norm_dataset_256.pickle') 

test_width_64 = 64
data_64 = pd.read_pickle('/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/real/francois_norm_dataset_64.pickle') 

In [9]:
import azure.ai.ml._artifacts._artifact_utilities as artifact_utils

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/final/SW_CNN_Trans.ckpt" # SW 1d CNN Transformer
SW_CNNTrans_detector_512 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/final/SW_adaFCN_Trans.ckpt" # SW ada 1d CNN Transformer
SW_adaCNNTrans_detector_512 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/final/SW_CNN_Dense.ckpt" # SW 1d CNN Transformer
SW_CNNDense_detector_512 = SlidingWindowLinearDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/final/SW_adaFCN_Dense.ckpt" # SW ada 1d CNN Transformer
SW_adaCNNDense_detector_512 = SlidingWindowLinearDetector.load_from_checkpoint(paths_SW).cpu(),

####
paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/SlidingWindowTransformerDetector_793.73K_04-06-2024_13:36:53/epoch=0-step=6500.ckpt" # SW ada 1d CNN Transformer FT val norm FP 0.2
SW_adaCNNTrans_512_0 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/SlidingWindowTransformerDetector_793.73K_04-06-2024_13:56:18/epoch=0-step=5000.ckpt" # SW ada 1d CNN Transformer FT val norm FP 0
SW_adaCNNTrans_512_1 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/SlidingWindowTransformerDetector_793.73K_04-06-2024_14:11:22/epoch=0-step=8000.ckpt" # SW ada 1d CNN Transformer FT val small FP 0
SW_adaCNNTrans_512_2 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/SlidingWindowTransformerDetector_793.73K_04-06-2024_14:25:47/epoch=0-step=5500.ckpt" # SW ada 1d CNN Transformer FT val small FP 0.2
SW_adaCNNTrans_512_3 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

/opt/miniconda/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'act_fct' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['act_fct'])`.
  rank_zero_warn(


In [10]:
def baseline_detector(input: torch.Tensor) -> int:   
    input.squeeze(0)
    prediction = 0

    center = int(input.shape[1]/2)
    # flag points with very high increment as artifact
    # Calculate increments by subtracting the tensor shifted by one from the original tensor
    increments = (input[0][1:] - input[0][:-1]).abs()
    mean_increment = torch.mean(increments)
    std_increment = torch.std(increments)

    if increments[center-1] > (mean_increment + 2*std_increment):
        prediction = 1


    return prediction

In [11]:
preds_baseline = list()
gt = list()

for sample in data_512:
    example = sample["data"]
    window  = test_width_512
    length  = len(example)

    example_data = torch.tensor(example)
    prediction_baseline = baseline_detector(example_data.unsqueeze(0))
    preds_baseline = preds_baseline + [prediction_baseline]


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, confusion_matrix, fbeta_score
import pandas as pd

metrics = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'mse'])
index = 0

preds_FCNTrans = list()
preds_adaFCNTrans = list()
preds_FCNDense = list()
preds_adaFCNDense = list()

preds_baseline = list()
gt = list()

for sample in data_512:
    example = sample["data"]
    stride  = 64
    window  = test_width_512
    length  = len(example)

    # add artifact to data
    example_data = torch.tensor(example)

    prediction_FCNTrans = SW_CNNTrans_detector_512.eval()(example_data.unsqueeze(0))   
    prediction_adaFCNTrans = SW_adaCNNTrans_detector_512.eval()(example_data.unsqueeze(0))
    prediction_FCNDense = SW_CNNDense_detector_512.eval()(example_data.unsqueeze(0))   
    prediction_adaFCNDense = SW_adaCNNDense_detector_512.eval()(example_data.unsqueeze(0))

    prediction_baseline = baseline_detector(example_data.unsqueeze(0))

    preds_FCNTrans = preds_FCNTrans + [prediction_FCNTrans.numpy()]
    preds_adaFCNTrans = preds_adaFCNTrans + [prediction_adaFCNTrans.numpy()]
    preds_FCNDense = preds_FCNDense + [prediction_FCNDense.numpy()]
    preds_adaFCNDense = preds_adaFCNDense + [prediction_adaFCNDense.numpy()]
    
    preds_baseline = preds_baseline + [prediction_baseline]

    gt = gt + [1]


/opt/miniconda/lib/python3.10/site-packages/torch/nn/modules/conv.py:303: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:883.)
  return F.conv1d(input, weight, bias, self.stride,


In [12]:
best_threshold_fbeta_FCNTrans_512 = 0.252
best_threshold_fbeta_adaFCNTrans_512 = 0.343

best_threshold_fbeta_FCNDense_512 = 0.343
best_threshold_fbeta_adaFCNDense_512 = 0.373

In [12]:
preds_binary_FCNTrans = [1 if value >= best_threshold_fbeta_FCNTrans_512 else 0 for value in preds_FCNTrans]
preds_binary_adaFCNTrans = [1 if value >= best_threshold_fbeta_adaFCNTrans_512 else 0 for value in preds_adaFCNTrans]
preds_binary_FCNDense = [1 if value >= best_threshold_fbeta_FCNDense_512 else 0 for value in preds_FCNDense]
preds_binary_adaFCNDense = [1 if value >= best_threshold_fbeta_adaFCNDense_512 else 0 for value in preds_adaFCNDense]

preds_binary_baseline = preds_baseline

tn_cnn_1, fp_cnn_1, fn_cnn_1, tp_cnn_1 = confusion_matrix(gt, preds_binary_FCNTrans, labels=[0, 1]).ravel()
tn_cnn_2, fp_cnn_2, fn_cnn_2, tp_cnn_2 = confusion_matrix(gt, preds_binary_adaFCNTrans, labels=[0, 1]).ravel()
tn_cnn_3, fp_cnn_3, fn_cnn_3, tp_cnn_3 = confusion_matrix(gt, preds_binary_FCNDense, labels=[0, 1]).ravel()
tn_cnn_4, fp_cnn_4, fn_cnn_4, tp_cnn_4 = confusion_matrix(gt, preds_binary_adaFCNDense, labels=[0, 1]).ravel()

tn_baseline, fp_baseline, fn_baseline, tp_baseline = confusion_matrix(gt, preds_binary_baseline, labels=[0, 1]).ravel()

metrics = pd.DataFrame([{
    'detector': 'FCN Trans',
    'threshold': best_threshold_fbeta_FCNTrans_512,
    'fbeta': fbeta_score(gt, preds_binary_FCNTrans, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_FCNTrans),
    'precision': precision_score(gt, preds_binary_FCNTrans),
    'recall': recall_score(gt, preds_binary_FCNTrans),
    'mse': mean_squared_error(gt, preds_binary_FCNTrans),
    'tn': tn_cnn_1,
    'fp': fp_cnn_1, 
    'fn': fn_cnn_1, 
    'tp': tp_cnn_1,
},
{
    'detector': 'adapted FCN Trans',
    'threshold': best_threshold_fbeta_adaFCNTrans_512,
    'fbeta': fbeta_score(gt, preds_binary_adaFCNTrans, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_adaFCNTrans),
    'precision': precision_score(gt, preds_binary_adaFCNTrans),
    'recall': recall_score(gt, preds_binary_adaFCNTrans),
    'mse': mean_squared_error(gt, preds_binary_adaFCNTrans),
    'tn': tn_cnn_2,
    'fp': fp_cnn_2, 
    'fn': fn_cnn_2, 
    'tp': tp_cnn_2
},
{
    'detector': 'FCN Dense',
    'threshold': best_threshold_fbeta_FCNDense_512,
    'fbeta': fbeta_score(gt, preds_binary_FCNDense, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_FCNDense),
    'precision': precision_score(gt, preds_binary_FCNDense),
    'recall': recall_score(gt, preds_binary_FCNDense),
    'mse': mean_squared_error(gt, preds_binary_FCNDense),
    'tn': tn_cnn_3,
    'fp': fp_cnn_3, 
    'fn': fn_cnn_3, 
    'tp': tp_cnn_3
},
{
    'detector': 'adapted FCN Dense',
    'threshold': best_threshold_fbeta_adaFCNDense_512,
    'fbeta': fbeta_score(gt, preds_binary_adaFCNDense, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_adaFCNDense),
    'precision': precision_score(gt, preds_binary_adaFCNDense),
    'recall': recall_score(gt, preds_binary_adaFCNDense),
    'mse': mean_squared_error(gt, preds_binary_adaFCNDense),
    'tn': tn_cnn_4,
    'fp': fp_cnn_4, 
    'fn': fn_cnn_4, 
    'tp': tp_cnn_4
},
{
    'detector': 'baseline',
    'threshold': 0.5,
    'fbeta': fbeta_score(gt, preds_binary_baseline, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_baseline),
    'precision': precision_score(gt, preds_binary_baseline),
    'recall': recall_score(gt, preds_binary_baseline),
    'mse': mean_squared_error(gt, preds_binary_baseline),
    'tn': tn_baseline,
    'fp': fp_baseline, 
    'fn': fn_baseline, 
    'tp': tp_baseline
}])

/opt/miniconda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
metrics

,detector,threshold,fbeta,accuracy,precision,recall,mse,tn,fp,fn,tp
0,FCN Trans,0.252,0.440000,0.135802,1.0,0.135802,0.864198,0,0,70,11
1,adapted FCN Trans,0.343,0.807860,0.456790,1.0,0.456790,0.543210,0,0,44,37
2,FCN Dense,0.343,0.247525,0.061728,1.0,0.061728,0.938272,0,0,76,5
3,adapted FCN Dense,0.373,0.112360,0.024691,1.0,0.024691,0.975309,0,0,79,2
4,baseline,0.500,0.000000,0.000000,0.0,0.000000,1.000000,0,0,81,0


## Models fine tuned on adapted Artifacts

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, confusion_matrix, fbeta_score
import pandas as pd

metrics = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'mse'])
index = 0

preds_FCNTrans = list()
preds_adaFCNTrans = list()
preds_FCNDense = list()
preds_adaFCNDense = list()

preds_baseline = list()
gt = list()

for sample in data_512:
    example = sample["data"]
    stride  = 64
    window  = test_width_512
    length  = len(example)

    # add artifact to data
    example_data = torch.tensor(example)

    prediction_FCNTrans = SW_adaCNNTrans_512_0.eval()(example_data.unsqueeze(0))   
    prediction_adaFCNTrans = SW_adaCNNTrans_512_1.eval()(example_data.unsqueeze(0))
    prediction_FCNDense = SW_adaCNNTrans_512_2.eval()(example_data.unsqueeze(0))   
    prediction_adaFCNDense = SW_adaCNNTrans_512_3.eval()(example_data.unsqueeze(0))

    prediction_baseline = baseline_detector(example_data.unsqueeze(0))

    preds_FCNTrans = preds_FCNTrans + [prediction_FCNTrans.numpy()]
    preds_adaFCNTrans = preds_adaFCNTrans + [prediction_adaFCNTrans.numpy()]
    preds_FCNDense = preds_FCNDense + [prediction_FCNDense.numpy()]
    preds_adaFCNDense = preds_adaFCNDense + [prediction_adaFCNDense.numpy()]
    
    preds_baseline = preds_baseline + [prediction_baseline]

    gt = gt + [1]


In [19]:
preds_binary_FCNTrans = [1 if value >= best_threshold_fbeta_adaFCNTrans_512 else 0 for value in preds_FCNTrans]
preds_binary_adaFCNTrans = [1 if value >= best_threshold_fbeta_adaFCNTrans_512 else 0 for value in preds_adaFCNTrans]
preds_binary_FCNDense = [1 if value >= best_threshold_fbeta_adaFCNTrans_512 else 0 for value in preds_FCNDense]
preds_binary_adaFCNDense = [1 if value >= best_threshold_fbeta_adaFCNTrans_512 else 0 for value in preds_adaFCNDense]

preds_binary_baseline = preds_baseline

tn_cnn_1, fp_cnn_1, fn_cnn_1, tp_cnn_1 = confusion_matrix(gt, preds_binary_FCNTrans, labels=[0, 1]).ravel()
tn_cnn_2, fp_cnn_2, fn_cnn_2, tp_cnn_2 = confusion_matrix(gt, preds_binary_adaFCNTrans, labels=[0, 1]).ravel()
tn_cnn_3, fp_cnn_3, fn_cnn_3, tp_cnn_3 = confusion_matrix(gt, preds_binary_FCNDense, labels=[0, 1]).ravel()
tn_cnn_4, fp_cnn_4, fn_cnn_4, tp_cnn_4 = confusion_matrix(gt, preds_binary_adaFCNDense, labels=[0, 1]).ravel()

tn_baseline, fp_baseline, fn_baseline, tp_baseline = confusion_matrix(gt, preds_binary_baseline, labels=[0, 1]).ravel()

metrics = pd.DataFrame([{
    'detector': 'adapted FCN Trans, old val file, fp 0.2',
    'threshold': best_threshold_fbeta_adaFCNTrans_512,
    'fbeta': fbeta_score(gt, preds_binary_FCNTrans, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_FCNTrans),
    'precision': precision_score(gt, preds_binary_FCNTrans),
    'recall': recall_score(gt, preds_binary_FCNTrans),
    'mse': mean_squared_error(gt, preds_binary_FCNTrans),
    'tn': tn_cnn_1,
    'fp': fp_cnn_1, 
    'fn': fn_cnn_1, 
    'tp': tp_cnn_1,
},
{
    'detector': 'adapted FCN Trans, old val file, fp 0',
    'threshold': best_threshold_fbeta_adaFCNTrans_512,
    'fbeta': fbeta_score(gt, preds_binary_adaFCNTrans, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_adaFCNTrans),
    'precision': precision_score(gt, preds_binary_adaFCNTrans),
    'recall': recall_score(gt, preds_binary_adaFCNTrans),
    'mse': mean_squared_error(gt, preds_binary_adaFCNTrans),
    'tn': tn_cnn_2,
    'fp': fp_cnn_2, 
    'fn': fn_cnn_2, 
    'tp': tp_cnn_2
},
{
    'detector': 'adapted FCN Trans, new val file, fp 0',
    'threshold': best_threshold_fbeta_adaFCNTrans_512,
    'fbeta': fbeta_score(gt, preds_binary_FCNDense, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_FCNDense),
    'precision': precision_score(gt, preds_binary_FCNDense),
    'recall': recall_score(gt, preds_binary_FCNDense),
    'mse': mean_squared_error(gt, preds_binary_FCNDense),
    'tn': tn_cnn_3,
    'fp': fp_cnn_3, 
    'fn': fn_cnn_3, 
    'tp': tp_cnn_3
},
{
    'detector': 'adapted FCN Trans, new val file, fp 0.2',
    'threshold': best_threshold_fbeta_adaFCNTrans_512,
    'fbeta': fbeta_score(gt, preds_binary_adaFCNDense, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_adaFCNDense),
    'precision': precision_score(gt, preds_binary_adaFCNDense),
    'recall': recall_score(gt, preds_binary_adaFCNDense),
    'mse': mean_squared_error(gt, preds_binary_adaFCNDense),
    'tn': tn_cnn_4,
    'fp': fp_cnn_4, 
    'fn': fn_cnn_4, 
    'tp': tp_cnn_4
},
{
    'detector': 'baseline',
    'threshold': 0.5,
    'fbeta': fbeta_score(gt, preds_binary_baseline, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_baseline),
    'precision': precision_score(gt, preds_binary_baseline),
    'recall': recall_score(gt, preds_binary_baseline),
    'mse': mean_squared_error(gt, preds_binary_baseline),
    'tn': tn_baseline,
    'fp': fp_baseline, 
    'fn': fn_baseline, 
    'tp': tp_baseline
}])

/opt/miniconda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
metrics

,detector,threshold,fbeta,accuracy,precision,recall,mse,tn,fp,fn,tp
0,"adapted FCN Trans, old val file, fp 0.2",0.343,0.353982,0.098765,1.0,0.098765,0.901235,0,0,73,8
1,"adapted FCN Trans, old val file, fp 0",0.343,0.285714,0.074074,1.0,0.074074,0.925926,0,0,75,6
2,"adapted FCN Trans, new val file, fp 0",0.343,0.488722,0.160494,1.0,0.160494,0.839506,0,0,68,13
3,"adapted FCN Trans, new val file, fp 0.2",0.343,0.353982,0.098765,1.0,0.098765,0.901235,0,0,73,8
4,baseline,0.500,0.000000,0.000000,0.0,0.000000,1.000000,0,0,81,0


## Adapted Model

In [30]:
val_path = Path("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/va_noCinCECGT_Francois64.pkl")
val = CachedArtifactDataset(file=val_path)

val_path = Path("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/val_noETTm_Francois64.pkl")
val_ETTm = CachedArtifactDataset(file=val_path)


In [31]:
paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/francois/SW_FCN_Trans_64.ckpt" # SW 1d CNN Transformer
SW_CNNTrans_detector_64 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/francois/SW_adaFCN_Trans_64.ckpt" # SW ada 1d CNN Transformer
SW_adaCNNTrans_detector_64 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/francois/SW_FCN_Trans_64_ETTm.ckpt" # SW ada 1d CNN Transformer
SW_adaCNNTrans_detector_64_ETTm = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

/opt/miniconda/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'act_fct' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['act_fct'])`.
  rank_zero_warn(


In [24]:
# val_path = Path("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/val_noETTm_256.pkl")
# val_ETTm = CachedArtifactDataset(file=val_path)

In [25]:
# paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/francois/SW_adaFCNTrans_256_noETTm_adaArt.ckpt" # SW ada 1d CNN Transformer
# SW_adaCNNTrans_detector_256_ETTm = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

# paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/francois/SW_adaFCNTrans_256_noETTm.ckpt" # SW ada 1d CNN Transformer
# SW_adaCNNTrans_detector_256_ETTm_adaArt = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, confusion_matrix
import pandas as pd

SW_detectors = [SW_CNNTrans_detector_64, SW_adaCNNTrans_detector_64]
# SW_detectors = [SW_adaCNNTrans_detector_64_ETTm]
# SW_detectors = [SW_adaCNNTrans_detector_256_ETTm, SW_adaCNNTrans_detector_256_ETTm_adaArt]

all_predictions_valSet = pd.DataFrame(columns=['Detector_id', 'predictions'])
index = 0
gt = list()

for detector in SW_detectors:

    preds = list()

    for sample in val_ETTm:
        example = sample["data"]
        window  = detector.window
        length  = len(example)

        # add artifact to data
        example_data = torch.tensor(example + sample["artifact"])

        # set detector to evaluation mode
        detector.eval()
        # make prediction and insert into prediction
        prediction = detector(example_data.unsqueeze(0))

        # update count
        preds = preds + [prediction.numpy()]

        if index == 0 :
            gt = gt + [sample["label"]]
    

    new_row = pd.DataFrame([{
        'Detector_id': index +1,
        'predictions': preds
    }])

    all_predictions_valSet = pd.concat([all_predictions_valSet,new_row])
    index = index +1
    print(len(all_predictions_valSet))

all_predictions_valSet = pd.concat([
    all_predictions_valSet,
    pd.DataFrame([{
        'Detector_id': 0,
        'predictions': gt
    }])
    ])

1
2


In [33]:
from sklearn.metrics import fbeta_score

gt = all_predictions_valSet.iloc[-1]["predictions"]
val_metrics = pd.DataFrame()
for index, row in all_predictions_valSet.iterrows():

    max_fbeta = 0

    for threshold in np.linspace(0,  1,  100):
        predictions = np.where(np.array(row['predictions']) > threshold, 1, 0)

        fbeta = fbeta_score(gt, predictions, average='macro', beta=0.5)

        if (fbeta > max_fbeta):
            max_fbeta = fbeta
            best_threshold_fbeta = threshold

    predictions = np.where(np.array(row['predictions']) > best_threshold_fbeta, 1, 0)

    tn, fp, fn, tp = confusion_matrix(gt, predictions, labels=[0, 1]).ravel()

    metric = pd.DataFrame([{
        'index': index,
        'detector': f"detector{row['Detector_id']}",
        'threshold': best_threshold_fbeta,
        'fbeta_score': fbeta_score(gt, predictions, beta=0.5),
        'accuracy': accuracy_score(gt, predictions),
        'precision': precision_score(gt, predictions),
        'recall': recall_score(gt, predictions),
        'mse': mean_squared_error(gt, predictions), 
        'tn': tn,
        'fp': fp, 
        'fn': fn, 
        'tp': tp
    }])

    val_metrics = pd.concat([val_metrics, metric], ignore_index=True)


In [34]:
val_metrics

,index,detector,threshold,fbeta_score,accuracy,precision,recall,mse,tn,fp,fn,tp
0,0,detector1,0.292929,0.921250,0.906738,0.935870,0.867069,0.093262,996,59,132,861
1,0,detector2,0.454545,0.931712,0.904785,0.957569,0.840886,0.095215,1018,37,158,835
2,0,detector0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1055,0,0,993


In [37]:
# best_threshold_fbeta_adaFCNTrans_64_ETTm = val_metrics["threshold"][0]

In [36]:
best_threshold_fbeta_FCNTrans_64 = val_metrics["threshold"][0]
best_threshold_fbeta_adaFCNTrans_64 = val_metrics["threshold"][1]

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, confusion_matrix, fbeta_score
import pandas as pd

metrics = pd.DataFrame()
index = 0

preds_FCNTrans = list()
preds_adaFCNTrans = list()
preds_adaFCNTrans_ETTm = list()

preds_baseline = list()
gt = list()

for sample in data_64:
    example = sample["data"]
    window  = test_width_64
    length  = len(example)

    # add artifact to data
    example_data = torch.tensor(example)

    prediction_FCNTrans = SW_CNNTrans_detector_64.eval()(example_data.unsqueeze(0))   
    prediction_adaFCNTrans = SW_adaCNNTrans_detector_64.eval()(example_data.unsqueeze(0))
    prediction_adaFCNTrans_ETTm = SW_adaCNNTrans_detector_64_ETTm.eval()(example_data.unsqueeze(0))

    prediction_baseline = baseline_detector(example_data.unsqueeze(0))

    preds_FCNTrans = preds_FCNTrans + [prediction_FCNTrans.numpy()]
    preds_adaFCNTrans = preds_adaFCNTrans + [prediction_adaFCNTrans.numpy()]
    preds_adaFCNTrans_ETTm = preds_adaFCNTrans_ETTm + [prediction_adaFCNTrans_ETTm.numpy()]

    preds_baseline = preds_baseline + [prediction_baseline]

    gt = gt + [1]


In [19]:
preds_binary_FCNTrans = [1 if value >= best_threshold_fbeta_FCNTrans_64 else 0 for value in preds_FCNTrans]
preds_binary_adaFCNTrans = [1 if value >= best_threshold_fbeta_adaFCNTrans_64 else 0 for value in preds_adaFCNTrans]
preds_binary_adaFCNTrans_ETTm = [1 if value >= best_threshold_fbeta_adaFCNTrans_64_ETTm else 0 for value in preds_adaFCNTrans_ETTm]

preds_binary_baseline = preds_baseline

tn_cnn_1, fp_cnn_1, fn_cnn_1, tp_cnn_1 = confusion_matrix(gt, preds_binary_FCNTrans, labels=[0, 1]).ravel()
tn_cnn_2, fp_cnn_2, fn_cnn_2, tp_cnn_2 = confusion_matrix(gt, preds_binary_adaFCNTrans, labels=[0, 1]).ravel()
tn_cnn_3, fp_cnn_3, fn_cnn_3, tp_cnn_3 = confusion_matrix(gt, preds_binary_adaFCNTrans_ETTm, labels=[0, 1]).ravel()

tn_baseline, fp_baseline, fn_baseline, tp_baseline = confusion_matrix(gt, preds_binary_baseline, labels=[0, 1]).ravel()

metrics = pd.DataFrame([{
    'detector': 'FCN Trans',
    'threshold': best_threshold_fbeta_FCNTrans_64,
    'fbeta': fbeta_score(gt, preds_binary_FCNTrans, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_FCNTrans),
    'precision': precision_score(gt, preds_binary_FCNTrans),
    'recall': recall_score(gt, preds_binary_FCNTrans),
    'mse': mean_squared_error(gt, preds_binary_FCNTrans),
    'tn': tn_cnn_1,
    'fp': fp_cnn_1, 
    'fn': fn_cnn_1, 
    'tp': tp_cnn_1,
},
{
    'detector': 'adapted FCN Trans',
    'threshold': best_threshold_fbeta_adaFCNTrans_64,
    'fbeta': fbeta_score(gt, preds_binary_adaFCNTrans, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_adaFCNTrans),
    'precision': precision_score(gt, preds_binary_adaFCNTrans),
    'recall': recall_score(gt, preds_binary_adaFCNTrans),
    'mse': mean_squared_error(gt, preds_binary_adaFCNTrans),
    'tn': tn_cnn_2,
    'fp': fp_cnn_2, 
    'fn': fn_cnn_2, 
    'tp': tp_cnn_2
},
{
    'detector': 'adapted FCN Trans Val ETTm',
    'threshold': best_threshold_fbeta_adaFCNTrans_64_ETTm,
    'fbeta': fbeta_score(gt, preds_binary_adaFCNTrans_ETTm, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_adaFCNTrans_ETTm),
    'precision': precision_score(gt, preds_binary_adaFCNTrans_ETTm),
    'recall': recall_score(gt, preds_binary_adaFCNTrans_ETTm),
    'mse': mean_squared_error(gt, preds_binary_adaFCNTrans_ETTm),
    'tn': tn_cnn_3,
    'fp': fp_cnn_3, 
    'fn': fn_cnn_3, 
    'tp': tp_cnn_3
},
{
    'detector': 'baseline',
    'threshold': 0.5,
    'fbeta': fbeta_score(gt, preds_binary_baseline, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_baseline),
    'precision': precision_score(gt, preds_binary_baseline),
    'recall': recall_score(gt, preds_binary_baseline),
    'mse': mean_squared_error(gt, preds_binary_baseline),
    'tn': tn_baseline,
    'fp': fp_baseline, 
    'fn': fn_baseline, 
    'tp': tp_baseline
}])

/opt/miniconda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
metrics

,detector,threshold,fbeta,accuracy,precision,recall,mse,tn,fp,fn,tp
0,FCN Trans,0.313131,0.384615,0.111111,1.0,0.111111,0.888889,0,0,72,9
1,adapted FCN Trans,0.454545,0.161290,0.037037,1.0,0.037037,0.962963,0,0,78,3
2,adapted FCN Trans Val ETTm,0.282828,0.321101,0.086420,1.0,0.086420,0.913580,0,0,74,7
3,baseline,0.500000,0.000000,0.000000,0.0,0.000000,1.000000,0,0,81,0


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, confusion_matrix, fbeta_score
import pandas as pd

metrics = pd.DataFrame()
index = 0

preds_FCNTrans = list()
preds_adaFCNTrans = list()

preds_baseline = list()
gt = list()

for sample in data_256:
    example = sample["data"]
    window  = test_width_64
    length  = len(example)

    # add artifact to data
    example_data = torch.tensor(example)

    prediction_FCNTrans = SW_adaCNNTrans_detector_256_ETTm.eval()(example_data.unsqueeze(0))   
    prediction_adaFCNTrans = SW_adaCNNTrans_detector_256_ETTm_adaArt.eval()(example_data.unsqueeze(0))

    prediction_baseline = baseline_detector(example_data.unsqueeze(0))

    preds_FCNTrans = preds_FCNTrans + [prediction_FCNTrans.numpy()]
    preds_adaFCNTrans = preds_adaFCNTrans + [prediction_adaFCNTrans.numpy()]

    preds_baseline = preds_baseline + [prediction_baseline]

    gt = gt + [1]


In [23]:
preds_binary_FCNTrans = [1 if value >= best_threshold_fbeta_FCNTrans_64 else 0 for value in preds_FCNTrans]
preds_binary_adaFCNTrans = [1 if value >= best_threshold_fbeta_adaFCNTrans_64 else 0 for value in preds_adaFCNTrans]

preds_binary_baseline = preds_baseline

tn_cnn_1, fp_cnn_1, fn_cnn_1, tp_cnn_1 = confusion_matrix(gt, preds_binary_FCNTrans, labels=[0, 1]).ravel()
tn_cnn_2, fp_cnn_2, fn_cnn_2, tp_cnn_2 = confusion_matrix(gt, preds_binary_adaFCNTrans, labels=[0, 1]).ravel()

tn_baseline, fp_baseline, fn_baseline, tp_baseline = confusion_matrix(gt, preds_binary_baseline, labels=[0, 1]).ravel()

metrics = pd.DataFrame([{
    'detector': 'FCN Trans',
    'threshold': best_threshold_fbeta_FCNTrans_64,
    'fbeta': fbeta_score(gt, preds_binary_FCNTrans, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_FCNTrans),
    'precision': precision_score(gt, preds_binary_FCNTrans),
    'recall': recall_score(gt, preds_binary_FCNTrans),
    'mse': mean_squared_error(gt, preds_binary_FCNTrans),
    'tn': tn_cnn_1,
    'fp': fp_cnn_1, 
    'fn': fn_cnn_1, 
    'tp': tp_cnn_1,
},
{
    'detector': 'adapted FCN Trans',
    'threshold': best_threshold_fbeta_adaFCNTrans_64,
    'fbeta': fbeta_score(gt, preds_binary_adaFCNTrans, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_adaFCNTrans),
    'precision': precision_score(gt, preds_binary_adaFCNTrans),
    'recall': recall_score(gt, preds_binary_adaFCNTrans),
    'mse': mean_squared_error(gt, preds_binary_adaFCNTrans),
    'tn': tn_cnn_2,
    'fp': fp_cnn_2, 
    'fn': fn_cnn_2, 
    'tp': tp_cnn_2
},
{
    'detector': 'baseline',
    'threshold': 0.5,
    'fbeta': fbeta_score(gt, preds_binary_baseline, beta=0.5),
    'accuracy': accuracy_score(gt, preds_binary_baseline),
    'precision': precision_score(gt, preds_binary_baseline),
    'recall': recall_score(gt, preds_binary_baseline),
    'mse': mean_squared_error(gt, preds_binary_baseline),
    'tn': tn_baseline,
    'fp': fp_baseline, 
    'fn': fn_baseline, 
    'tp': tp_baseline
}])

/opt/miniconda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
metrics

,detector,threshold,fbeta,accuracy,precision,recall,mse,tn,fp,fn,tp
0,FCN Trans,0.515152,0.488722,0.160494,1.0,0.160494,0.839506,0,0,68,13
1,adapted FCN Trans,0.525253,0.605096,0.234568,1.0,0.234568,0.765432,0,0,62,19
2,baseline,0.500000,0.000000,0.000000,0.0,0.000000,1.000000,0,0,81,0
